<a href="https://colab.research.google.com/github/blooming-ai/generativeai/blob/main/generativeai/examples/generative/ipynb/gpt_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT code
https://github.com/karpathy/minGPT

In [1]:
#Get data file
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-31 04:37:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-07-31 04:37:56 (18.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



## Word embedding

In [31]:
import re
import pdb

#Byte pair encoding
def construct_vocab(vocab:dict, file_path:str)->dict:
  '''
  Read file and update vocab dict. vocab has format vocab[('w','o','r','d')]->freq
  '''
  with open(file_path) as fp:
    while True:
      line = fp.readline()
      if not line: break
      else:
        lst = line.split()
        for item in lst:
          item.strip()
          item = "".join(ch for ch in item if ch.isalnum()) # remove special characters
          lst_item = tuple(item.lower()) # tuple to make a hashable dict key
          if len(lst_item) == 0: continue #ignore empty key
          if lst_item not in vocab: vocab[lst_item] = 1
          else:                     vocab[lst_item] += 1

  return vocab

def get_byte_pair_hist(vocab:dict)->dict:
  '''
  Read vocab[('w','o','r','d')]->freq and construct byte pair histogram
  returns pair[('w','o')]->freq
  '''
  pair = {}
  for word, freq in vocab.items():
    for i in range(len(word)-1):
      key = (word[i],word[i+1])
      if key not in pair: pair[key] = freq
      else:               pair[key] += freq

  return pair

def merge_pair(replace_pair:tuple, vocab_in:dict)->dict:
  '''
  replace pair in the key of the vocab. E.g. pair=('w','o') then update
  vocab_in[('w','o','r','d')]->freq to
  vocab_in[('wo','r','d')]->freq
  '''
  vocab_out = {}
  for word, freq in vocab_in.items():
    word_new = list()
    is_last_char_used = False
    for i in range(len(word)-1):
      if (word[i],word[i+1]) == replace_pair:
        word_new.append(word[i]+word[i+1])
        if i == len(word)-2: is_last_char_used = True
      else:
        word_new.append(word[i])

    print(word)
    if not is_last_char_used : word_new.append(word[len(word)-1])

    vocab_out[tuple(word_new)] = freq
  return vocab_out

vocab = defaultdict()
file_path = "sample_data/input.txt"
vocab = construct_vocab(vocab, file_path)
print(() in vocab)
pairs = get_byte_pair_hist(vocab)
best = max(pairs, key=pairs.get)
# vocab_out = merge_pair(('f','i'), vocab)

print(vocab)
print(pairs)
print(best)
# print(vocab_out)

True
defaultdict(None, {('f', 'i', 'r', 's', 't'): 361, ('c', 'i', 't', 'i', 'z', 'e', 'n'): 100, ('b', 'e', 'f', 'o', 'r', 'e'): 193, ('w', 'e'): 865, ('p', 'r', 'o', 'c', 'e', 'e', 'd'): 21, ('a', 'n', 'y'): 189, ('f', 'u', 'r', 't', 'h', 'e', 'r'): 36, ('h', 'e', 'a', 'r'): 230, ('m', 'e'): 1764, ('s', 'p', 'e', 'a', 'k'): 292, ('a', 'l', 'l'): 890, ('y', 'o', 'u'): 3142, ('a', 'r', 'e'): 784, ('r', 'e', 's', 'o', 'l', 'v', 'e', 'd'): 15, ('r', 'a', 't', 'h', 'e', 'r'): 77, ('t', 'o'): 4766, ('d', 'i', 'e'): 138, ('t', 'h', 'a', 'n'): 480, ('f', 'a', 'm', 'i', 's', 'h'): 4, ('k', 'n', 'o', 'w'): 350, ('c', 'a', 'i', 'u', 's'): 17, ('m', 'a', 'r', 'c', 'i', 'u', 's'): 122, ('i', 's'): 2078, ('c', 'h', 'i', 'e', 'f'): 7, ('e', 'n', 'e', 'm', 'y'): 42, ('t', 'h', 'e'): 6283, ('p', 'e', 'o', 'p', 'l', 'e'): 88, ('k', 'n', 'o', 'w', 't'): 6, ('l', 'e', 't'): 437, ('u', 's'): 399, ('k', 'i', 'l', 'l'): 49, ('h', 'i', 'm'): 1206, ('a', 'n', 'd'): 5680, ('w', 'e', 'l', 'l'): 563, ('h', 'a',

## Positional Encoding

In [ ]:
#Sinusoidal position encoding

## Transformer

In [18]:
from collections import defaultdict
def def_value():
  '''
  Function to return a default values for a key that is not present in
  defaultdict
  '''
  return 0
a = 'aa'
di = defaultdict()
di[a] += 1
print(di)

KeyError: ignored

## Network Setup

## Load Data

# Train